In [ ]:
# Inicialize o Otter
import otter
grader = otter.Notebook("hw11.ipynb")

# Lição de Casa 11: Inferência de Regressão

**Recurso Útil:**

- [Referência Python](http://data8.org/sp22/python-reference.html): Folha de dicas de métodos úteis de array e tabela usados no Data 8!

**Leitura Recomendada**: 

* [Usando Intervalos de Confiança](https://inferentialthinking.com/chapters/13/4/Using_Confidence_Intervals.html)
* [A Linha de Regressão](https://inferentialthinking.com/chapters/15/2/Regression_Line.html#the-regression-line-in-standard-units)
* [Inferência para Regressão](https://www.inferentialthinking.com/chapters/16/Inference_for_Regression.html)

Por favor, complete este caderno preenchendo as células fornecidas. Antes de começar, execute a célula a seguir para configurar o caderno importando algumas bibliotecas úteis. Toda vez que você iniciar seu servidor, precisará executar esta célula novamente.

Para todos os problemas nos quais você deve escrever explicações e frases, você **deve** fornecer sua resposta no espaço designado. **Além disso, ao longo desta lição de casa e de todas as futuras, certifique-se de não reatribuir variáveis ao longo do caderno!** Por exemplo, se você usar `max_temperature` em sua resposta para uma questão, não a reatribua posteriormente. Caso contrário, você falhará em testes que pensava ter passado anteriormente!

**Nota: Esta lição de casa possui testes ocultos. Isso significa que, mesmo que os testes digam que 100% passaram, não significa que sua nota final será 100%. Nós rodaremos mais testes para correção uma vez que todos entregarem a lição de casa.**

Compartilhar respostas diretamente não é permitido, mas discutir problemas com a equipe do curso ou com outros estudantes é encorajado.

Você deve começar cedo para que tenha tempo de obter ajuda se estiver com dificuldades.

In [ ]:
# Não altere esta célula; apenas execute-a.

import numpy as np
from datascience import *
import d8error

# Estas linhas fazem alguma mágica de plotagem sofisticada
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore')  # Ignora avisos simples
from datetime import datetime

## Uma Introdução à Inferência de Regressão

Anteriormente nesta disciplina, usamos intervalos de confiança para quantificar a incerteza sobre estimativas. Também podemos realizar testes de hipóteses usando um intervalo de confiança seguindo o procedimento abaixo:

1. Definir uma hipótese nula e uma hipótese alternativa (elas devem ser da forma "O parâmetro é X" e "O parâmetro não é X").
2. Escolher um corte de valor-p e chamá-lo de $q$.
3. Construir um intervalo de $(100-q)\%$ usando amostragem bootstrap (por exemplo, se seu corte de valor-p é 0,01, ou 1%, então construa um intervalo de confiança de 99%).
4. Usando o intervalo de confiança, determinar se seus dados são mais consistentes com sua hipótese nula ou alternativa:
   * Se o parâmetro X da hipótese nula estiver no seu intervalo de confiança, os dados são mais consistentes com a hipótese nula.
   * Se o parâmetro X da hipótese nula *não* estiver no seu intervalo de confiança, os dados são mais consistentes com a hipótese alternativa.

Mais recentemente, discutimos o uso da regressão linear para fazer previsões baseadas em variáveis correlacionadas. Por exemplo, podemos prever a altura das crianças com base nas alturas de seus pais.

Podemos combinar esses dois tópicos para fazer declarações poderosas sobre nossa população usando as seguintes técnicas:

- Intervalo bootstrap para a verdadeira inclinação
- Intervalo de previsão bootstrap para y (dado um valor particular de x)

Esta lição de casa explora esses dois métodos.

## Os Dados
O [Batuíra-de-coleira](https://www.audubon.org/field-guide/bird/snowy-plover) é uma pequena ave que vive na costa em partes da Califórnia e em outros lugares. É tão pequeno que é vulnerável a muitos predadores, incluindo pessoas e cães que não olham por onde andam quando vão à praia. É considerado em perigo em muitas partes dos EUA.

Os dados são sobre os ovos e filhotes recém-nascidos da Batuíra-de-coleira. Aqui está uma foto de [uma ave adulta incubando seus ovos](http://cescos.fau.edu/jay/eps/articles/snowyplover.html).

<img src="plover_and_eggs.jpeg" alt="Batuíra-de-coleira e Ovos">

Os dados foram coletados no Parque Nacional de Point Reyes por um ex-[estudante de Berkeley](https://openlibrary.org/books/OL2038693M/BLSS_the_Berkeley_interactive_statistical_system). O objetivo era ver como o tamanho de um ovo poderia ser usado para prever o peso do filhote resultante. Quanto maior o filhote recém-nascido, maior a probabilidade de sobreviver.

<img src="plover_and_chick.jpeg" alt="Batuíra-de-coleira e Filhote">

Cada linha da tabela abaixo corresponde a um ovo de Batuíra-de-coleira e o filhote resultante. Note como a ave é pequena:

- `Egg Length` (Comprimento do Ovo) e `Egg Breadth` (Largura do ovo, diâmetro mais largo) são medidos em milímetros
- `Egg Weight` (Peso do Ovo) e `Bird Weight` (Peso do Pássaro) são medidos em gramas; para comparação, um clipe de papel padrão pesa cerca de uma grama

In [ ]:
birds = Table.read_table('snowy_plover.csv')
birds

Nesta investigação, usaremos o peso do ovo para prever o peso do pássaro. Execute a célula abaixo para criar um gráfico de dispersão dos pesos dos ovos e dos pesos dos pássaros, juntamente com sua linha de melhor ajuste.

In [ ]:
# Apenas execute esta célula e examine o gráfico de dispersão.
birds.scatter('Egg Weight', "Bird Weight", fit_line=True)

## 1. Encontrando o Intervalo de Confiança Bootstrap para a Inclinação Verdadeira

Olhando para o gráfico de dispersão da nossa amostra, observamos uma relação linear entre o peso do ovo e o peso do pássaro. No entanto, relações que aparecem em uma amostra podem não existir na população da qual a amostra foi retirada.

Queremos saber se realmente existe uma relação linear entre o peso do ovo e o peso do pássaro para as Batuíras-de-coleira. Se não houver uma relação linear entre as duas variáveis, então esperaríamos uma correlação de 0. Consequentemente, a inclinação da linha de regressão também seria 0.

<!-- BEGIN QUESTION -->

**Questão 1.1.** Vamos realizar um teste de hipótese usando intervalos de confiança para verificar se existe uma relação linear entre o peso do ovo e o peso do pássaro. Defina as hipóteses nula e alternativa que permitirão conduzir este teste. **(8 pontos)**

*Nota:* Por favor, escreva sua resposta **na célula abaixo** no seguinte formato:
- **Hipótese Nula:**
- **Hipótese Alternativa:**



_Digite sua resposta aqui, substituindo este texto._

**Questão 1.2.** Defina as seguintes duas funções:

1. `standard_units`: Esta função recebe um array de números e retorna um array contendo esses números convertidos para unidades padrão.
2. `correlation`: Esta função recebe uma tabela e dois nomes de colunas (um para *x* e outro para *y*) e retorna a correlação entre essas colunas.

**(8 pontos)**

In [ ]:
def standard_units(arr):
    ...

def correlation(tbl, x_col, y_col):
    ...

In [ ]:
grader.check("q1_2")

**Questão 1.3.** Usando as funções que você acabou de implementar, crie uma função chamada `fit_line`. Ela deve receber uma tabela como `birds` e os nomes das colunas associadas a *x* e *y* como seus argumentos e retornar um *array* contendo a inclinação e a intercepção da linha de regressão (nessa ordem) que prevê a coluna *y* na tabela usando a coluna *x*. **(8 pontos)**

In [ ]:
def fit_line(tbl, x_col, y_col):
    ...

fit_line(birds, "Egg Weight", "Bird Weight")

In [ ]:
grader.check("q1_3")

**Execute** esta célula para plotar a linha produzida ao chamar `fit_line` na tabela `birds`.

**Nota:** Você não é responsável pelo código na célula abaixo, mas certifique-se de que sua função `fit_line` gerou uma linha razoável para os dados.

In [ ]:
# Certifique-se de que sua função fit_line ajusta uma linha razoável
# aos dados em birds, usando o gráfico abaixo.

# Apenas execute esta célula
slope, intercept = fit_line(birds, "Egg Weight", "Bird Weight")
birds.scatter("Egg Weight", "Bird Weight")
plt.plot([min(birds.column("Egg Weight")), max(birds.column("Egg Weight"))], 
         [slope*min(birds.column("Egg Weight"))+intercept, slope*max(birds.column("Egg Weight"))+intercept])
plt.show()

Agora temos todas as ferramentas de que precisamos para criar um intervalo de confiança que quantifica nossa incerteza sobre a verdadeira relação entre o peso do ovo e o peso do pássaro.

<!-- BEGIN QUESTION -->

**Questão 1.4.** Crie um array chamado `resampled_slopes` que contém a inclinação da melhor linha de ajuste para 1000 reamostras bootstrap de `birds`. Plote a distribuição dessas inclinações. **(8 pontos)**

In [ ]:
resampled_slopes = ...

for i in np.arange(1000): 
    birds_bootstrap = ...
    bootstrap_line = ...
    bootstrap_slope = ...
    resampled_slopes = ...
    
# NÃO ALTERE ESTA LINHA
Table().with_column("Slope estimate", resampled_slopes).hist()

In [ ]:
grader.check("q1_4")

<!-- END QUESTION -->

**Questão 1.5.** Use suas inclinações reamostradas para construir um intervalo de confiança de 95% para o valor verdadeiro da inclinação. **(8 pontos)**


In [ ]:
lower_end = ...
upper_end = ...
print("95% confidence interval for slope: [{:g}, {:g}]".format(lower_end, upper_end))

In [ ]:
grader.check("q1_5")

<!-- BEGIN QUESTION -->

**Questão 1.6.** Baseado no seu intervalo de confiança, você aceitaria ou rejeitaria a hipótese nula de que a inclinação verdadeira é 0? Por quê? Qual é o corte de valor-p que você está usando? **(8 pontos)**


_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Questão 1.7.** Qual você acha que é a inclinação verdadeira? Você não precisa de um número exato. Quão confiante você está dessa estimativa? **(8 pontos)**

*Dica:* Você pode fornecer um intervalo no qual acredita que a inclinação verdadeira se encaixa?


_Digite sua resposta aqui, substituindo este texto._

<!-- END QUESTION -->

## 2. Encontrando o Intervalo de Previsão Bootstrap

Suponha que estamos visitando Point Reyes e nos deparamos com alguns ovos de Batuíra-de-coleira; gostaríamos de saber quão pesados eles serão quando eclodirem. Em outras palavras, queremos usar nossa linha de regressão para fazer previsões sobre o peso de um pássaro com base no peso do ovo correspondente.

No entanto, assim como estamos incertos sobre a inclinação da verdadeira linha de regressão, também estamos incertos sobre as previsões feitas com base na verdadeira linha de regressão.

**Questão 2.1.** Defina a função `fitted_value`. Ela deve receber quatro argumentos:

1. `table`: uma tabela como `birds`. Estaremos prevendo os valores na segunda coluna usando a primeira.
2. `x_col`: o nome da nossa coluna x dentro da `table` de entrada
3. `y_col`: o nome da nossa coluna y dentro da `table` de entrada
4. `given_x`: um número, o valor da variável preditora para a qual gostaríamos de fazer uma previsão.
  
A função deve retornar a previsão da linha para o x dado. **(8 pontos)**

*Dica:* Certifique-se de usar a função `fit_line` que você definiu na Questão 1.3.

In [ ]:
def fitted_value(table, x_col, y_col, given_x):
    line = ...
    slope = ...
    intercept = ...
    ...

# Aqui está um exemplo de como fitted_value é usado. O código abaixo
# calcula a previsão para o peso do pássaro, em gramas, baseado em
# um peso de ovo de 8 gramas.
egg_weight_eight = fitted_value(birds, "Egg Weight", "Bird Weight", 8)
egg_weight_eight

In [ ]:
grader.check("q2_1")

**Questão 2.2.** Raymond, o especialista residente em Batuíra-de-coleira em Point Reyes, nos informa que o ovo que ele tem observado cuidadosamente tem um peso de 9 gramas. Usando `fitted_value` acima, atribua a variável `experts_egg` ao peso previsto do pássaro para o ovo de Raymond.

In [ ]:
experts_egg = ...
experts_egg

In [ ]:
grader.check("q2_2")

In [ ]:
# Vamos olhar o número de linhas na tabela birds.
birds.num_rows

Um frequentador do parque levanta a seguinte objeção à sua previsão:

> "Sua previsão depende da sua amostra de 44 pássaros. Sua previsão mudaria se você tivesse uma amostra diferente de 44 pássaros?"

Tendo lido a seção [16.3](https://inferentialthinking.com/chapters/16/3/Prediction_Intervals.html) do livro didático, você sabe exatamente como responder! Se a amostra fosse diferente, a linha de regressão também seria diferente. Isso resultaria em uma previsão diferente. Para ver quão boa é nossa previsão, precisamos ter uma ideia de quão variável a previsão pode ser.

**Questão 2.3.** Defina uma função `compute_resampled_line` que recebe uma tabela `tbl` e dois nomes de colunas, `x_col` e `y_col`, e retorna um array contendo os parâmetros da melhor linha de ajuste (inclinação e interceptação) para uma reamostragem bootstrap da tabela.

In [ ]:
def compute_resampled_line(tbl, x_col, y_col):
    resample = ...
    resampled_line = ...
    ...

In [ ]:
grader.check("q2_3")

**Execute** a célula a seguir para definir a função `bootstrap_lines`. Ela recebe quatro argumentos:
1. `tbl`: uma tabela como `birds`
2. `x_col`: o nome da nossa coluna x dentro da tabela de entrada `tbl`
3. `y_col`: o nome da nossa coluna y dentro da tabela de entrada `tbl`
4. `num_bootstraps`: um inteiro, o número de reamostragens bootstrap a serem realizadas.

Ela retorna uma *tabela* com uma linha para cada reamostra bootstrap e as seguintes duas colunas:
1. `Slope`: as inclinações obtidas por bootstrap
2. `Intercept`: as interceptações correspondentes obtidas por bootstrap

In [ ]:
# Apenas execute esta célula
def bootstrap_lines(tbl, x_col, y_col, num_bootstraps):
    resampled_slopes = make_array()
    resampled_intercepts = make_array() 
    for i in np.arange(num_bootstraps): 
        resampled_line = compute_resampled_line(tbl, x_col, y_col) 
        resampled_slope = resampled_line.item(0) 
        resampled_intercept = resampled_line.item(1) 
        resampled_slopes = np.append(resampled_slopes,resampled_slope)
        resampled_intercepts = np.append(resampled_intercepts,resampled_intercept)
    tbl_lines = Table().with_columns('Slope', resampled_slopes, 'Intercept', resampled_intercepts)
    return tbl_lines

regression_lines = bootstrap_lines(birds, "Egg Weight", "Bird Weight", 1000)
regression_lines

<!-- BEGIN QUESTION -->

**Questão 2.4.** Crie um array chamado `predictions_for_eight` que contém os pesos previstos dos pássaros com base em um ovo de 8 gramas para cada linha de regressão em `regression_lines`. **(8 pontos)**


In [ ]:
predictions_for_eight = ...

# Isso criará um histograma das suas previsões:
table_of_predictions = Table().with_column('Predictions at Egg Weight=8', predictions_for_eight)
table_of_predictions.hist('Predictions at Egg Weight=8', bins=20)

In [ ]:
grader.check("q2_4")

<!-- END QUESTION -->

**Questão 2.5.** Crie um intervalo de confiança aproximado de 95% para essas previsões. **(6 pontos)**


In [ ]:
lower_bound = ...
upper_bound = ...

print('95% Confidence interval for predictions for x=8: (', lower_bound,",", upper_bound, ')')

In [ ]:
grader.check("q2_5")

**Questão 2.6.** Atribua `plover_statements` a um array de inteiro(s) que corresponda(m) a afirmação(ões) verdadeira(s). **(6 pontos)**

1. O intervalo de confiança de 95% cobre 95% dos pesos dos pássaros para ovos que tinham um peso de oito gramas em `birds`.

2. O intervalo de confiança de 95% dá uma ideia de quanto os pesos reais diferem da sua previsão.

3. O intervalo de confiança de 95% quantifica a incerteza na nossa estimativa do que a verdadeira linha preveria.

In [ ]:
plover_statements = ...

In [ ]:
grader.check("q2_6")

Você concluiu a Lição de Casa 11!

**Passos importantes para a submissão:**
1. Execute os testes e verifique se todos passaram.
2. Escolha **Salvar Caderno** no menu **Arquivo**, depois **execute a célula final**.
3. Clique no link para baixar o arquivo zip.
4. Então, submeta o arquivo zip para a tarefa correspondente de acordo com as instruções do seu instrutor.

**É sua responsabilidade garantir que seu trabalho esteja salvo antes de executar a última célula.**

## Submissão

Certifique-se de ter executado todas as células no seu caderno em ordem antes de executar a célula abaixo, para que todas as imagens/gráficos apareçam na saída. A célula abaixo irá gerar um arquivo zip para você submeter. **Por favor, salve antes de exportar!**

In [ ]:
# Salve seu caderno primeiro, depois execute esta célula para exportar sua submissão.
grader.export(pdf=False, run_tests=True)